In [ ]:
# Importación de bibliotecas para machine learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import (classification_report, confusion_matrix, 
                           accuracy_score, precision_score, recall_score, 
                           f1_score, roc_auc_score, roc_curve)
import time

# Intentar importar XGBoost y LightGBM
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except ImportError:
    XGB_AVAILABLE = False
    print("⚠️ XGBoost no disponible")

try:
    import lightgbm as lgb
    LGB_AVAILABLE = True
except ImportError:
    LGB_AVAILABLE = False
    print("⚠️ LightGBM no disponible")

warnings.filterwarnings('ignore')

print("✅ Bibliotecas de machine learning importadas correctamente")
print(f"📊 XGBoost disponible: {XGB_AVAILABLE}")
print(f"📊 LightGBM disponible: {LGB_AVAILABLE}")


In [ ]:
# Cargar los datos preprocesados
try:
    processed_datasets = joblib.load('../models/processed_datasets.joblib')
    label_encoder = joblib.load('../models/label_encoder.joblib')
    print("✅ Datos preprocesados cargados correctamente")
except FileNotFoundError:
    print("❌ Error: Ejecutar primero el notebook de preprocesamiento")
    print("📋 Archivos necesarios:")
    print("   • ../models/processed_datasets.joblib")
    print("   • ../models/label_encoder.joblib")

print(f"\n📊 DATASETS DISPONIBLES:")
for name in processed_datasets.keys():
    dataset = processed_datasets[name]
    print(f"   ✅ {name}:")
    print(f"      🏋️ Train: {dataset['X_train'].shape}")
    print(f"      🧪 Test: {dataset['X_test'].shape}")

print(f"\n🎯 CLASES DEL TARGET:")
for i, class_name in enumerate(label_encoder.classes_):
    print(f"   {i}: {class_name}")

# Seleccionar el dataset principal para benchmarking (StandardScaler + OneHot)
main_dataset_name = 'StandardScaler_OneHot'
main_dataset = processed_datasets[main_dataset_name]

X_train = main_dataset['X_train']
X_test = main_dataset['X_test']
y_train = main_dataset['y_train']
y_test = main_dataset['y_test']

print(f"\n🎯 DATASET PRINCIPAL SELECCIONADO: {main_dataset_name}")
print(f"   📊 Entrenamiento: {X_train.shape}")
print(f"   📊 Prueba: {X_test.shape}")


In [ ]:
# Definir los modelos para benchmarking
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42, max_depth=10),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Support Vector Machine': SVC(random_state=42, probability=True),
    'Naive Bayes': GaussianNB()
}

# Agregar XGBoost si está disponible
if XGB_AVAILABLE:
    models['XGBoost'] = xgb.XGBClassifier(random_state=42, eval_metric='mlogloss')

# Agregar LightGBM si está disponible
if LGB_AVAILABLE:
    models['LightGBM'] = lgb.LGBMClassifier(random_state=42, verbose=-1)

print("🤖 MODELOS DEFINIDOS PARA BENCHMARKING")
print("="*50)
for i, (name, model) in enumerate(models.items(), 1):
    print(f"   {i:2d}. {name}")
    print(f"       📋 Tipo: {type(model).__name__}")

print(f"\n📊 Total de modelos: {len(models)}")

# Configuración de validación cruzada
cv_folds = 5
cv_scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

print(f"\n🔄 CONFIGURACIÓN DE VALIDACIÓN CRUZADA:")
print(f"   📊 Número de folds: {cv_folds}")
print(f"   📈 Métricas de evaluación: {', '.join(cv_scoring)}")
print(f"   🎯 Estratificación: Activada (mantiene proporción de clases)")


In [ ]:
# Realizar validación cruzada para todos los modelos
cv_results = {}
training_times = {}

print("🔄 EJECUTANDO VALIDACIÓN CRUZADA")
print("="*40)

# Configurar validación cruzada estratificada
skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)

for model_name, model in models.items():
    print(f"\n🚀 Entrenando: {model_name}")
    
    # Medir tiempo de entrenamiento
    start_time = time.time()
    
    # Realizar validación cruzada para cada métrica
    cv_results[model_name] = {}
    
    for metric in cv_scoring:
        scores = cross_val_score(model, X_train, y_train, 
                               cv=skf, scoring=metric, n_jobs=-1)
        cv_results[model_name][metric] = {
            'scores': scores,
            'mean': scores.mean(),
            'std': scores.std()
        }
    
    # Guardar tiempo de entrenamiento
    end_time = time.time()
    training_times[model_name] = end_time - start_time
    
    print(f"   ⏱️ Tiempo: {training_times[model_name]:.2f}s")
    print(f"   📊 Accuracy: {cv_results[model_name]['accuracy']['mean']:.4f} (±{cv_results[model_name]['accuracy']['std']:.4f})")
    print(f"   📊 F1-Score: {cv_results[model_name]['f1_macro']['mean']:.4f} (±{cv_results[model_name]['f1_macro']['std']:.4f})")

print(f"\n✅ VALIDACIÓN CRUZADA COMPLETADA")
print(f"🎯 {len(models)} modelos evaluados con {cv_folds}-fold CV")


In [ ]:
# Crear tabla resumen de resultados
results_df = pd.DataFrame()

for model_name in cv_results.keys():
    row = {
        'Model': model_name,
        'Accuracy_Mean': cv_results[model_name]['accuracy']['mean'],
        'Accuracy_Std': cv_results[model_name]['accuracy']['std'],
        'Precision_Mean': cv_results[model_name]['precision_macro']['mean'],
        'Precision_Std': cv_results[model_name]['precision_macro']['std'],
        'Recall_Mean': cv_results[model_name]['recall_macro']['mean'],
        'Recall_Std': cv_results[model_name]['recall_macro']['std'],
        'F1_Mean': cv_results[model_name]['f1_macro']['mean'],
        'F1_Std': cv_results[model_name]['f1_macro']['std'],
        'Training_Time': training_times[model_name]
    }
    results_df = pd.concat([results_df, pd.DataFrame([row])], ignore_index=True)

# Ordenar por F1-Score promedio
results_df = results_df.sort_values('F1_Mean', ascending=False).reset_index(drop=True)

print("📊 RESULTADOS DE VALIDACIÓN CRUZADA")
print("="*80)
print(f"{'Rank':<4} {'Model':<20} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Time(s)':<8}")
print("-" * 80)

for i, row in results_df.iterrows():
    print(f"{i+1:<4} {row['Model']:<20} "
          f"{row['Accuracy_Mean']:.3f}±{row['Accuracy_Std']:.3f} "
          f"{row['Precision_Mean']:.3f}±{row['Precision_Std']:.3f} "
          f"{row['Recall_Mean']:.3f}±{row['Recall_Std']:.3f} "
          f"{row['F1_Mean']:.3f}±{row['F1_Std']:.3f} "
          f"{row['Training_Time']:<8.2f}")

# Identificar el mejor modelo
best_model_name = results_df.iloc[0]['Model']
best_f1_score = results_df.iloc[0]['F1_Mean']

print(f"\n🏆 MEJOR MODELO: {best_model_name}")
print(f"   📊 F1-Score: {best_f1_score:.4f}")
print(f"   ⏱️ Tiempo de entrenamiento: {results_df.iloc[0]['Training_Time']:.2f}s")


In [ ]:
# Visualización de resultados
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('📊 COMPARACIÓN DE MODELOS DE MACHINE LEARNING\n🎯 Resultados de Validación Cruzada', 
             fontsize=16, fontweight='bold', y=0.98)

# Subplot 1: Accuracy comparison
ax1 = axes[0,0]
accuracy_means = [cv_results[model]['accuracy']['mean'] for model in results_df['Model']]
accuracy_stds = [cv_results[model]['accuracy']['std'] for model in results_df['Model']]

bars1 = ax1.barh(results_df['Model'], accuracy_means, xerr=accuracy_stds, 
                 color='skyblue', alpha=0.8, capsize=5)
ax1.set_title('🎯 Accuracy Comparison', fontweight='bold', fontsize=12)
ax1.set_xlabel('Accuracy Score')
ax1.grid(True, alpha=0.3)

# Añadir valores en las barras
for i, (bar, mean_val) in enumerate(zip(bars1, accuracy_means)):
    ax1.text(mean_val + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{mean_val:.3f}', va='center', fontweight='bold')

# Subplot 2: F1-Score comparison
ax2 = axes[0,1]
f1_means = [cv_results[model]['f1_macro']['mean'] for model in results_df['Model']]
f1_stds = [cv_results[model]['f1_macro']['std'] for model in results_df['Model']]

bars2 = ax2.barh(results_df['Model'], f1_means, xerr=f1_stds, 
                 color='lightgreen', alpha=0.8, capsize=5)
ax2.set_title('📈 F1-Score Comparison', fontweight='bold', fontsize=12)
ax2.set_xlabel('F1-Score (Macro)')
ax2.grid(True, alpha=0.3)

# Añadir valores en las barras
for i, (bar, mean_val) in enumerate(zip(bars2, f1_means)):
    ax2.text(mean_val + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{mean_val:.3f}', va='center', fontweight='bold')

# Subplot 3: Training time comparison
ax3 = axes[1,0]
times = [training_times[model] for model in results_df['Model']]

bars3 = ax3.barh(results_df['Model'], times, color='lightcoral', alpha=0.8)
ax3.set_title('⏱️ Training Time Comparison', fontweight='bold', fontsize=12)
ax3.set_xlabel('Training Time (seconds)')
ax3.grid(True, alpha=0.3)

# Añadir valores en las barras
for bar, time_val in zip(bars3, times):
    ax3.text(time_val + max(times)*0.01, bar.get_y() + bar.get_height()/2, 
             f'{time_val:.2f}s', va='center', fontweight='bold')

# Subplot 4: Radar chart de métricas del mejor modelo
ax4 = axes[1,1]

# Preparar datos para radar chart
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
best_model_scores = [
    cv_results[best_model_name]['accuracy']['mean'],
    cv_results[best_model_name]['precision_macro']['mean'],
    cv_results[best_model_name]['recall_macro']['mean'],
    cv_results[best_model_name]['f1_macro']['mean']
]

# Crear gráfico polar
angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
scores = best_model_scores + [best_model_scores[0]]  # Cerrar el polígono
angles += angles[:1]

ax4 = plt.subplot(2, 2, 4, projection='polar')
ax4.plot(angles, scores, 'o-', linewidth=2, color='red', alpha=0.8)
ax4.fill(angles, scores, alpha=0.25, color='red')
ax4.set_xticks(angles[:-1])
ax4.set_xticklabels(metrics)
ax4.set_ylim(0, 1)
ax4.set_title(f'🏆 {best_model_name}\\nMétricas de Rendimiento', 
              fontweight='bold', fontsize=12, pad=20)

# Añadir valores en el radar
for angle, score, metric in zip(angles[:-1], best_model_scores, metrics):
    ax4.text(angle, score + 0.05, f'{score:.3f}', 
             ha='center', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Guardar tabla de resultados
results_df.to_csv('../reports/cv_results_comparison.csv', index=False)
print(f"\n💾 Resultados guardados: ../reports/cv_results_comparison.csv")


In [ ]:
# Entrenar el mejor modelo en todo el conjunto de entrenamiento
best_model = models[best_model_name]
best_model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

print(f"🏆 EVALUACIÓN FINAL DEL MEJOR MODELO: {best_model_name}")
print("="*60)

# Métricas en el conjunto de prueba
test_accuracy = accuracy_score(y_test, y_pred)
test_precision = precision_score(y_test, y_pred, average='macro')
test_recall = recall_score(y_test, y_pred, average='macro')
test_f1 = f1_score(y_test, y_pred, average='macro')

print(f"📊 MÉTRICAS EN CONJUNTO DE PRUEBA:")
print(f"   🎯 Accuracy: {test_accuracy:.4f}")
print(f"   📈 Precision (macro): {test_precision:.4f}")
print(f"   📈 Recall (macro): {test_recall:.4f}")
print(f"   📈 F1-Score (macro): {test_f1:.4f}")

# Comparar con resultados de validación cruzada
cv_accuracy = cv_results[best_model_name]['accuracy']['mean']
cv_f1 = cv_results[best_model_name]['f1_macro']['mean']

print(f"\n🔄 COMPARACIÓN CON VALIDACIÓN CRUZADA:")
print(f"   📊 Accuracy - CV: {cv_accuracy:.4f} | Test: {test_accuracy:.4f} | Diff: {abs(cv_accuracy-test_accuracy):.4f}")
print(f"   📊 F1-Score - CV: {cv_f1:.4f} | Test: {test_f1:.4f} | Diff: {abs(cv_f1-test_f1):.4f}")

# Guardar el modelo entrenado
model_filename = f'../models/best_model_{best_model_name.replace(" ", "_")}.joblib'
joblib.dump(best_model, model_filename)
print(f"\n💾 Mejor modelo guardado: {model_filename}")

# Preparar datos para visualización
test_results = {
    'model_name': best_model_name,
    'test_accuracy': test_accuracy,
    'test_precision': test_precision,
    'test_recall': test_recall,
    'test_f1': test_f1,
    'cv_accuracy': cv_accuracy,
    'cv_f1': cv_f1
}

# Guardar métricas
joblib.dump(test_results, '../models/test_results.joblib')
print(f"💾 Métricas de prueba guardadas: ../models/test_results.joblib")
